In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('diabetic_data.csv')

FileNotFoundError: File b'diabetic_data.csv' does not exist

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df['payer_code'].unique()

In [ ]:
df.medical_specialty.unique()

In [ ]:
df.columns

In [ ]:
df.groupby('max_glu_serum').count()

In [ ]:
#
medication = ['max_glu_serum', 'A1Cresult','metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone']

In [3]:
meds_df = df[medication]
for med in medication:
    print(med, meds_df[med].value_counts())

NameError: name 'df' is not defined

In [4]:
df.weight.value_counts()

NameError: name 'df' is not defined

In [5]:
df.readmitted.value_counts()

NameError: name 'df' is not defined

In [13]:
df.admission_type_id.value_counts()

1    53990
3    18869
2    18480
6     5291
5     4785
8      320
7       21
4       10
Name: admission_type_id, dtype: int64

# Mapping Admission, Discharge & Admission Source Types

In [14]:
mapping = pd.read_csv('IDs_mapping.csv')

In [15]:
admission_type_map = mapping[:8]
discharge_type_map = mapping[10:40]
admission_source_map = mapping[42:]
discharge_type_map.columns = ['discharge_disposition_id','description']
admission_source_map.columns = ['admission_source_id', 'description']

In [16]:
discharge_type_map

,discharge_disposition_id,description
10,1,Discharged to home
11,2,Discharged/transferred to another short term h...
12,3,Discharged/transferred to SNF
13,4,Discharged/transferred to ICF
14,5,Discharged/transferred to another type of inpa...
15,6,Discharged/transferred to home with home healt...
16,7,Left AMA
17,8,Discharged/transferred to home under care of H...
18,9,Admitted as an inpatient to this hospital
19,10,Neonate discharged to another hospital for neo...


# Cleaning Data

In [17]:
# Removing discharge codes 11 (Expired) ,13-14 (Hospice care), 19-21 (Expired)
terminally_ill = [11,13,14,19,21]
df = df[~df['discharge_disposition_id'].isin(terminally_ill)]

In [18]:
# Removing NaN & Not mapped values
df = df[~df['admission_type_id'].isin([6,8])]
df = df[~df['admission_source_id'].isin([17,20])]
df = df[~df['discharge_disposition_id'].isin([25])]

In [19]:
# Dropping encounter_id & medication with less than 2 varying values
df.drop(['encounter_id','metformin-pioglitazone','metformin-rosiglitazone',
         'glimepiride-pioglitazone','troglitazone','acetohexamide','examide','citoglipton'], axis =1, inplace= True)

In [20]:
# Replacing weight value '?' into 'Not Available'
df = df.replace(to_replace='?', value='NaN')

In [21]:
# Number of repeated visits for patients
(df.patient_nbr.value_counts() >1).sum()

14921

In [22]:
# Checking for NA values in target variable
df.readmitted.isna().sum() 

0

In [28]:
df.diag_1.value_counts()

428       6035
414       6030
786       3549
410       3131
486       3009
427       2427
491       2028
715       1873
780       1816
682       1809
996       1784
434       1771
276       1649
250.8     1495
599       1446
38        1362
584       1333
250.6     1093
V57       1064
493        972
820        957
577        927
435        927
562        887
518        860
574        854
296        800
250.7      792
250.13     790
440        765
          ... 
448          1
373          1
834          1
957          1
895          1
698          1
615          1
381          1
314          1
543          1
605          1
955          1
471          1
143          1
363          1
791          1
684          1
372          1
133          1
160          1
911          1
98           1
700          1
V51          1
365          1
97           1
84           1
817          1
691          1
412          1
Name: diag_1, Length: 711, dtype: int64

In [24]:
df.shape

(89984, 42)

In [30]:
df.medical_specialty.value_counts()

NaN                                     45505
InternalMedicine                        12718
Emergency/Trauma                         7396
Family/GeneralPractice                   5639
Cardiology                               4235
Surgery-General                          2512
Orthopedics                              1386
Nephrology                               1359
Radiologist                              1121
Orthopedics-Reconstructive                770
Psychiatry                                702
Pulmonology                               692
Surgery-Cardiovascular/Thoracic           622
ObstetricsandGynecology                   591
Urology                                   590
Surgery-Vascular                          499
Gastroenterology                          479
Surgery-Neuro                             438
PhysicalMedicineandRehabilitation         383
Oncology                                  317
Pediatrics                                241
Neurology                         

(1) Check through each diagnoses and map it to the ICD-9 Codes so that we can categorize the type of diagnoses and hopefully establish a pattern of comorbidity 

In [32]:
# set the values in our diagnoses columns as ICD codes
ICD_Codes_1 = df['diag_1'].value_counts()
ICD_Codes_2 = df['diag_2'].value_counts()
ICD_Codes_3 = df['diag_3'].value_counts()

(2) Group the medications into distinct families so that we can keep the medication data but can eliminate some of the features

(3) Do something about the medical specialty column and that can hopefully streamline the extensive list of medical professional categories 